In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from PIL import Image

# Assuming your data shape is (n_samples, 170, 5)
# where 170 = 17 Hz * 10 seconds, and 5 is the number of dimensions

def create_spectrograms(data, sample_rate=17):
    """
    Convert multi-dimensional time series to spectrograms
    
    Parameters:
    -----------
    data : array-like, shape (n_samples, time_steps, n_dimensions)
        Time series data
    sample_rate : int
        Sampling frequency in Hz
    
    Returns:
    --------
    spectrograms : array, shape (n_samples, freq_bins, time_bins, n_dimensions)
    """
    f, t, Sxx = signal.spectrogram(data, fs=17, nperseg=32, noverlap=16)
    return 10 * np.log10(np.array(Sxx)), f, t

def visualize_spectrograms(spectrograms, frequencies, times, reseize=False):    
    fig, axes = plt.subplots(1, spectrograms.shape[1], figsize=(20, 4))
    for i in range(spectrograms.shape[1]):
        if reseize:
            img = Image.fromarray(spectrograms[0, i, :, :])
            img = img.resize((224, 224), Image.Resampling.BICUBIC)
            img = np.array(img)
        else:
            img = spectrograms[0, i, :, :]

        im = axes[i].pcolormesh(np.arange(img.shape[1]), np.arange(img.shape[0]), img, 
                                shading='gouraud', cmap='viridis')
        axes[i].set_ylabel('Frequency [Hz]')
        axes[i].set_xlabel('Time [sec]')
        axes[i].set_title(f'Dimension {i+1}')
        plt.colorbar(im, ax=axes[i])
    plt.tight_layout()
    plt.show()

In [ ]:

# Example usage:
# Generate dummy data: 100 samples, 170 time steps, 5 dimensions
dummy_data = np.random.randn(100, 5, 170)

# Create spectrograms
spectrograms, frequencies, times = create_spectrograms(dummy_data)
print(f"Spectrogram shape: {spectrograms.shape}")
print(f"Frequencies shape: {frequencies.shape}")
print(f"Time bins shape: {times.shape}")

# Visualize spectrograms for the first sample
visualize_spectrograms(spectrograms, frequencies, times)

In [ ]:
print(frequencies)

In [ ]:
import pandas as pd

df = pd.read_csv("../../Multiple Distances Three Events.csv")
df.head()

In [ ]:
data = np.array(np.split(df.values, df.shape[0] // 170))
data.shape

In [ ]:
data[0, :2]

In [ ]:
spectrograms, frequencies, times = create_spectrograms(data.transpose(0, 2, 1))
print(f"Spectrogram shape: {spectrograms.shape}")
print(f"Frequencies shape: {frequencies.shape}")
print(f"Time bins shape: {times.shape}")

In [ ]:
spectrograms.min(), spectrograms.max(), spectrograms.mean(), spectrograms.std()

In [ ]:
visualize_spectrograms(spectrograms, frequencies, times)

In [ ]:
resized_spectrograms = np.empty((spectrograms.shape[0], 224, 224, spectrograms.shape[3]))
for i in range(spectrograms.shape[0]):
    for j in range(spectrograms.shape[3]):
        img = Image.fromarray(spectrograms[i, :, :, j])
        img = img.resize((224, 224), Image.Resampling.BICUBIC)
        resized_spectrograms[i, :, :, j] = np.array(img)

In [ ]:
resized_spectrograms.shape

In [ ]:
labels = pd.read_csv("../../Labels Three Events.csv").values.flatten()

In [ ]:
X = spectrograms.reshape(spectrograms.shape[0], -1) 

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
y_train.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
pd.read_csv("../Labels Three Events.csv").value_counts()

In [ ]:
resized_spectrograms = np.empty((spectrograms.shape[0], 224, 224, spectrograms.shape[3]))
for i in range(spectrograms.shape[0]):
    for j in range(spectrograms.shape[3]):
        img = Image.fromarray(spectrograms[i, :, :, j])
        img = img.resize((224, 224), Image.Resampling.BICUBIC)
        resized_spectrograms[i, :, :, j] = np.array(img)